In [ ]:
#This file is designed to export gas data of the snapshots into a file that can be used to create plots with gas in the background
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import os
import pickle
import matplotlib
import matplotlib.colors as colors
import gizmo_analysis as gizmo
import astropy
from astropy.io import ascii
#############################################################################
#constants
#############################################################################
MsunToGm = 1.99e33
KpcToCm = 3.086e21
mp = 1.67e-24
#bin_edge = 10.
bin_edge = 30.
bins = np.arange(-25,25,0.1)

# Setting simulation parameters

In [ ]:
############################################################################
#read in sim files and find relevant particles
############################################################################
simname = 'm12i_res7100_mhdcv'
simtype="fire2"  #this is the simtype eg. fire2, fire3, sf-fire3, sf-fire3-alpha01, sf-fire3-alpha03, sf-fire3-alpha05 
simdir = '/scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12i_r7100/mhdcv/1Myr/fire2/'
snapnumber=596      #this is the true snapshot number each time for eg, 596, 597 .. and son
#reading in snapshot_times.txt file to get snapshot numbers and times


#reading in snapshot_times.txt file to get snapshot numbers and times
# columns are: snapshot scale-factor redshift time[Gyr] time_width[Myr]
snapshot_times = simdir + 'snapshot_times.txt'
snaptime_data = astropy.io.ascii.read(snapshot_times, guess=False, comment="#")
snaptime_data = np.genfromtxt(snapshot_times, usecols=(0,3), skip_header=4, dtype=float) #the first and fourth columns are the only ones we need 
snaps = np.array(snaptime_data[:,0]) #col1 = first column saved from text file #This is a collection of all snapshot nos.
times = np.array(snaptime_data[:,1]) #col4 = second column saved #This is a collection of times equivalent to those snapshot nos.

#######################################
#######################################

# Exporting components of gas plot to a file

In [ ]:
gas_datapath="./"
tracked_gas={} 
snaptime = times[np.where(snaps == snapnumber)][0] #time of snapshot in Gyr

part = gizmo.io.Read.read_snapshots(['all'],'snapshot_index', snapnumber, simulation_name=simname, simulation_directory=simdir, assign_hosts_rotation=True, assign_hosts=True)  
#t = np.max(part['star'].prop('form.time'))  

rGas = part['gas'].prop('host.distance.principal.cylindrical')[:,0]

xGas = part['gas'].prop('host.distance.principal.cartesian')[:,0]
yGas = part['gas'].prop('host.distance.principal.cartesian')[:,1]
zGas = part['gas'].prop('host.distance.principal.cartesian')[:,2]


mGas = part['gas']['mass']
rhoGas = part['gas']['density']
tGas = part['gas']['temperature']
idGas = part['gas']['id']

i_gas = np.where((rGas <= bin_edge) & (np.fabs(zGas) <= 1.5) & (part['gas']['density']*((MsunToGm/KpcToCm**3)/mp) >= 10.) & (tGas <= 1e4))

x = xGas[i_gas]
y = yGas[i_gas]
z = zGas[i_gas]

rho = part['gas'].prop('number.density')[i_gas]
id = part['gas']['id'][i_gas]

###########################################################################
#gas image (2d histogram)
###########################################################################
#cold (< 10^4 K) gas in the midplane (|z| <= 1.5 kpc within bin_edge
v =  np.where((rGas <= bin_edge) & (np.fabs(zGas) <= 1.5) & (tGas <= 1e4))
face, xh, yh = np.histogram2d(part['gas'].prop('host.distance.principal.cartesian')[v,1][0],part['gas'].prop('host.distance.principal.cartesian')[v,0][0],bins=[bins,bins], weights=part['gas']['mass'][v])

###########################################################################
tracked_gas={"snaptime":snaptime,"v":v,"face":face,"xh":xh,"yh":yh}
file_name=simtype+"_gas_data"+str(snapnumber)+".pkl"
with open(gas_datapath+file_name, 'wb') as output:
    pickle.dump(tracked_gas, output)
print("\n Stored the gas data for background plot from the snapshot no.",snapnumber,"to filename:",file_name,"\n#####\n")

#Copy this code block from below to another cell and read the file if you want the generate plot fromt the file
'''
This is how to read the data:
file_name="gas_data_snapshot_"+str(snapnumber)+".pkl"
with open(gas_datapath+file_name, "rb") as input:
  import_gasdata = pickle.load(input
  
#Extracting the variables  
v=import_gasdata["v"]
face=import_gasdata["face"]
xh=import_gasdata["xh"]
yh=import_gasdata["yh"]
#Use these as follows to generate the plot if you are going to generate plot using the above exported pkl file
'''

# Creating the plot

In [ ]:
from matplotlib import pyplot as plt
fig1=plt.figure()
fig1.set_size_inches(7,7)
ax=fig1.add_axes([0.17, 0.185, 0.65, 0.65]) #left, bottom, width, height
bin_edge = 30.
#bins = np.arange(-5,5,0.1)
bins = np.arange(-25,25,0.1)
norm = matplotlib.colors.LogNorm(vmin=1, vmax=1000) #the color range plotted
im = ax.imshow(face/(((bins[1]-bins[0])*1000)**2),origin='lower',interpolation='nearest',norm=norm,extent=(-25,25,-25,25),cmap='binary') 

#colorbar for the background gas density
cmap_gray = matplotlib.cm.get_cmap('binary')
norm1 = matplotlib.colors.LogNorm(vmin=1,vmax=1000)
cbar_ax1 = fig1.add_axes([0.04, 0.185, 0.04, 0.64]) # position of gray colorbar (left, bottom, width, height)
cb1 = fig1.colorbar(im, cax=cbar_ax1, ticklocation='left')
cb1.set_label('$\Sigma$ (M$_{{\odot}}$/pc$^2$)', labelpad=-5, fontsize=12)
#plot a scale bar 5 kpc long
ax.plot([-22.5,-17.5], [22.5,22.5], 'k-', linewidth=5)
label1 = "5 kpc"
ax.text(-22.5, 20, label1, fontsize=12.5)